In [1]:
import keras

Using TensorFlow backend.
/home/jared/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [2]:
%cd data

data_dir = %pwd
train_dir = data_dir + '/train/'
valid_dir = data_dir + '/valid/'
test_dir = data_dir + '/test/'
sample_dir = data_dir + '/sample/'

/home/jared/courses/deeplearning2/Kaggle/fisheries/data


In [3]:
import os
from glob import glob
import numpy as np
from shutil import copyfile

num_sample = 5

In [4]:
from keras.models import Sequential
from keras.layers import Dense, BatchNormalization, Dropout, Flatten, Convolution2D, MaxPooling2D, Input
from keras.optimizers import Adam

#base_dir = sample_dir
path = data_dir + '/'
batch_size = 64
image_size = (224,224)
image_shape = (*image_size, 3)

classes = ['ALB', 'BET', 'DOL', 'LAG', 'NoF', 'OTHER', 'SHARK', 'YFT']

In [11]:
for cls in classes:
    files = glob(train_dir + cls + '/*.jpg')
    
    num_valid = int(len(files) * .2)
    
    np.random.shuffle(files)
    
    os.mkdir(sample_dir + 'train/' + cls)
    for file in files[:num_sample]:
        copyfile(file, sample_dir + 'train/' + cls + '/' + os.path.basename(file))
    
    os.mkdir(sample_dir + 'valid/' + cls)
    for file in files[num_sample:num_sample+num_sample]:
        copyfile(file, sample_dir + 'valid/' + cls + '/' + os.path.basename(file))
    
    os.mkdir(valid_dir + cls)
    for file in files[:num_valid]:
        os.rename(file, valid_dir + cls + '/' + os.path.basename(file))
    

In [5]:
from keras.preprocessing.image import ImageDataGenerator

gen = ImageDataGenerator()
val_gen = ImageDataGenerator()

batches = gen.flow_from_directory(path+'train/', target_size=image_size, batch_size=batch_size, shuffle=True)
val_batches = val_gen.flow_from_directory(path+'valid/', target_size=image_size, batch_size=batch_size, shuffle=True)

train_steps = (batches.n//batch_size) + 1
val_steps = (val_batches.n//batch_size) + 1

Found 3025 images belonging to 8 classes.
Found 752 images belonging to 8 classes.


# Simple Model

In [6]:
model = Sequential([
    BatchNormalization(input_shape=image_shape)
    , Convolution2D(64, (3,3), activation='relu')
    , MaxPooling2D((2,2))
    , Convolution2D(128, (3,3), activation='relu')
    , MaxPooling2D((2,2))
    , Flatten()
    , Dense(1024, activation='relu')
    , Dense(8, activation='softmax')
])

model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
batch_normalization_1 (Batch (None, 224, 224, 3)       12        
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 222, 222, 64)      1792      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 111, 111, 64)      0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 109, 109, 128)     73856     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 54, 54, 128)       0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 373248)            0         
_________________________________________________________________
dense_1 (Dense)              (None, 1024)              382206976 
__________

In [ ]:
model.optimizer.lr = 0.00001
model.fit_generator(batches, train_steps, epochs=1, validation_data=val_batches, validation_steps=val_steps)

Epoch 1/1
